# Solutions 4 - Age of Recommendation

After __Age of Search__ (see the Page Rank notebook), we are now in the 
__Age of Recommendation__. This notebook is about Netflix Recommendations using 
Simon Funk's algorithm as implemented in [IncrementalSVD.jl](https://github.com/aaw/IncrementalSVD.jl) by Aaron Windsor.

## Messages

Easy -> hard?

Present: __age of search__ -> _mathematics_

Future: __age of recommendation__  -> _mathematics_

(BigData, new technologies)


## Easy -> hard

weight:  1 _kg_  -> 1 _kg_ $\pm$ 0.000000001 _kg_

running: 100 *m* -> 42,195 *m* ili 100 *m* < 10 *sek*

mathematics: exam -> state competition  -> [Olympiad](http://www.imo-official.org/)

search, recommending: good -> excellent

## Age of Search

google (and others)


* [50 billion pages](http://www.worldwidewebsize.com/), [3.5 billion querries daily](http://www.internetlivestats.com/google-search-statistics/)
* __PageRank__
* history, context - cookies, storing data (about you), [200+ parameters](http://backlinko.com/google-ranking-factors)

## Age of Recommendation

NetFlix, Amazon Prime, PickBox, ... - on-line streaming of movies and shows

[NetFlix](https://www.netflix.com/hr/)

 * [182 million users](https://www.statista.com/statistics/250934/quarterly-number-of-netflix-streaming-subscribers-worldwide/), 5,000 movies
 * [NetFlix Prize](http://www.kdd.org/kdd2014/tutorials/KDD%20-%20The%20Recommender%20Problem%20Revisited.pdf)
 

## Mathematics

Netflix Recommendation Engine is based on approximation of a (large and sparse) matrix
```
M = Users x Movies 
```
using (approximation of) singular value decomposition (SVD): 

* [IncrementalSVD.jl](https://github.com/aaw/IncrementalSVD.jl)
* [A parallel recommendation engine in Julia](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html)

## Similarities

Similarity of users $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[i,:],M[k,:])}{\|M[i,:]\| \cdot \|M[k,:]\|}
$$
Similarity of movies $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[:,i],M[:,k)}{\|M[:,i]\| \cdot \|M[:,k]\|}
$$

## Search

Row $M[u,:]$ - what user $u$ thinks about movies

Column $M[:,m]$ - what users think about movie $m$

Element $M[u,m]$ - what user $u$ thinks about movie $m$.

## Problem

Matrix $M$ is sparse so we do not have enough information. For example, 

```
900188 marks / ( 6040 users x 3706 movies ) = 4%
```

## Approximation

SVD decomposition $M=U\Sigma V^T$ is [approximated by a low-rank matrix](https://en.wikipedia.org/wiki/Low-rank_approximation) (for example. $rank=25$)

![SVD decomposition](svd.png)

The approximation matrix is __full__ and __gives enough good information__.

Prize for efficient approximation algorithm was $\$$ 1.000.000.  

In [1]:
# using Pkg
# Pkg.add(PackageSpec(url="https://github.com/aaw/IncrementalSVD.jl"))
# or
# pkg> add https://github.com/aaw/IncrementalSVD.jl

In [1]:
using IncrementalSVD

┌ Info: Precompiling IncrementalSVD [48ceb036-d4ca-4e51-bf5d-01664867517d]
└ @ Base loading.jl:1260
┌ Warning: InfoZIP falling back to ZipFile.jl backend!
└ @ InfoZIP C:\Users\Ivan_Slapnicar\.julia\packages\InfoZIP\3AT7m\src\InfoZIP.jl:20
┌ Warning: Package InfoZIP does not have ZipFile in its dependencies:
│ - If you have InfoZIP checked out for development and have
│   added ZipFile as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with InfoZIP
└ Loading ZipFile into InfoZIP from project dependency, future warnings for InfoZIP are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for IncrementalSVD [48ceb036-d4ca-4e51-bf5d-01664867517d]
│   exception = ErrorException("Required dependency InfoZIP [f4508453-b816-52ab-a864-26fc7f6211fc] failed to load from a cache file.")
└ @ Base loading.jl:1041
┌ Info: Precompiling InfoZIP [f4508453-b816-52a

In [2]:
varinfo(IncrementalSVD)

| name                         |        size | summary                              |
|:---------------------------- | -----------:|:------------------------------------ |
| IncrementalSVD               | 174.490 KiB | Module                               |
| Rating                       |   208 bytes | DataType                             |
| RatingSet                    |   216 bytes | DataType                             |
| RatingsModel                 |   224 bytes | DataType                             |
| cosine_similarity            |     0 bytes | typeof(cosine_similarity)            |
| get_predicted_rating         |     0 bytes | typeof(get_predicted_rating)         |
| item_features                |     0 bytes | typeof(item_features)                |
| item_search                  |     0 bytes | typeof(item_search)                  |
| items                        |     0 bytes | typeof(items)                        |
| load_book_crossing_dataset   |     0 bytes | typeof(load_book_crossing_dataset)   |
| load_large_movielens_dataset |     0 bytes | typeof(load_large_movielens_dataset) |
| load_small_movielens_dataset |     0 bytes | typeof(load_small_movielens_dataset) |
| rmse                         |     0 bytes | typeof(rmse)                         |
| similar_items                |     0 bytes | typeof(similar_items)                |
| similar_users                |     0 bytes | typeof(similar_users)                |
| split_ratings                |     0 bytes | typeof(split_ratings)                |
| train                        |     0 bytes | typeof(train)                        |
| truncate_model!              |     0 bytes | typeof(truncate_model!)              |
| user_features                |     0 bytes | typeof(user_features)                |
| user_ratings                 |     0 bytes | typeof(user_ratings)                 |
| users                        |     0 bytes | typeof(users)                        |


In [3]:
rating_set = load_small_movielens_dataset()

Reusing existing downloaded files...


Loading ratings 100%|███████████████████████████████████| Time: 0:00:02


RatingSet(Rating[Rating(3555, 1026, 4.0f0), Rating(4541, 1868, 2.0f0), Rating(1272, 1069, 1.0f0), Rating(308, 1980, 4.0f0), Rating(3012, 24, 2.0f0), Rating(4559, 1345, 4.0f0), Rating(4064, 817, 3.0f0), Rating(3195, 1062, 3.0f0), Rating(4808, 2740, 3.0f0), Rating(4984, 1300, 5.0f0)  …  Rating(3526, 1088, 5.0f0), Rating(500, 1532, 2.0f0), Rating(5005, 387, 4.0f0), Rating(2683, 79, 5.0f0), Rating(2180, 681, 3.0f0), Rating(624, 601, 4.0f0), Rating(1941, 1459, 4.0f0), Rating(1223, 886, 1.0f0), Rating(118, 1081, 5.0f0), Rating(2484, 1614, 4.0f0)], Rating[Rating(1491, 344, 2.0f0), Rating(3089, 45, 5.0f0), Rating(2986, 1559, 3.0f0), Rating(1491, 223, 1.0f0), Rating(4725, 568, 3.0f0), Rating(424, 619, 3.0f0), Rating(714, 402, 4.0f0), Rating(1698, 772, 3.0f0), Rating(1139, 825, 3.0f0), Rating(3705, 1658, 4.0f0)  …  Rating(4265, 763, 5.0f0), Rating(1794, 60, 4.0f0), Rating(5597, 636, 5.0f0), Rating(3445, 1292, 3.0f0), Rating(1996, 49, 5.0f0), Rating(4277, 1523, 4.0f0), Rating(1679, 825, 3.0f0), R

In [4]:
propertynames(rating_set)

(:training_set, :test_set, :user_to_index, :item_to_index)

In [5]:
# The format is (user, movie, mark)
rating_set.training_set

900188-element Array{Rating,1}:
 Rating(3555, 1026, 4.0f0)
 Rating(4541, 1868, 2.0f0)
 Rating(1272, 1069, 1.0f0)
 Rating(308, 1980, 4.0f0)
 Rating(3012, 24, 2.0f0)
 Rating(4559, 1345, 4.0f0)
 Rating(4064, 817, 3.0f0)
 Rating(3195, 1062, 3.0f0)
 Rating(4808, 2740, 3.0f0)
 Rating(4984, 1300, 5.0f0)
 Rating(3792, 1526, 3.0f0)
 Rating(376, 339, 5.0f0)
 Rating(4808, 82, 3.0f0)
 ⋮
 Rating(3280, 615, 3.0f0)
 Rating(2967, 2157, 2.0f0)
 Rating(3526, 1088, 5.0f0)
 Rating(500, 1532, 2.0f0)
 Rating(5005, 387, 4.0f0)
 Rating(2683, 79, 5.0f0)
 Rating(2180, 681, 3.0f0)
 Rating(624, 601, 4.0f0)
 Rating(1941, 1459, 4.0f0)
 Rating(1223, 886, 1.0f0)
 Rating(118, 1081, 5.0f0)
 Rating(2484, 1614, 4.0f0)

In [6]:
rating_set.test_set

100021-element Array{Rating,1}:
 Rating(1491, 344, 2.0f0)
 Rating(3089, 45, 5.0f0)
 Rating(2986, 1559, 3.0f0)
 Rating(1491, 223, 1.0f0)
 Rating(4725, 568, 3.0f0)
 Rating(424, 619, 3.0f0)
 Rating(714, 402, 4.0f0)
 Rating(1698, 772, 3.0f0)
 Rating(1139, 825, 3.0f0)
 Rating(3705, 1658, 4.0f0)
 Rating(4144, 721, 2.0f0)
 Rating(5249, 1313, 2.0f0)
 Rating(4506, 632, 5.0f0)
 ⋮
 Rating(2038, 815, 5.0f0)
 Rating(187, 293, 4.0f0)
 Rating(4265, 763, 5.0f0)
 Rating(1794, 60, 4.0f0)
 Rating(5597, 636, 5.0f0)
 Rating(3445, 1292, 3.0f0)
 Rating(1996, 49, 5.0f0)
 Rating(4277, 1523, 4.0f0)
 Rating(1679, 825, 3.0f0)
 Rating(3320, 936, 5.0f0)
 Rating(216, 1299, 3.0f0)
 Rating(3358, 92, 3.0f0)

In [7]:
# Users and their IDs
rating_set.user_to_index

Dict{AbstractString,Int32} with 6040 entries:
  "4304" => 4304
  "3935" => 3935
  "5422" => 5422
  "5734" => 5734
  "2243" => 2243
  "1881" => 1881
  "5425" => 5425
  "4209" => 4209
  "1907" => 1907
  "2923" => 2923
  "599"  => 599
  "2491" => 2491
  "5944" => 5944
  "228"  => 228
  "2590" => 2590
  "3697" => 3697
  "5031" => 5031
  "2579" => 2579
  "5551" => 5551
  "1880" => 1880
  "2562" => 2562
  "3215" => 3215
  "3991" => 3991
  "4652" => 4652
  "4088" => 4088
  ⋮      => ⋮

In [8]:
# Movies and their IDs
rating_set.item_to_index

Dict{AbstractString,Int32} with 3706 entries:
  "Fried Green Tomatoes (1991)"                                       => 594
  "Milk Money (1994)"                                                 => 1361
  "From Russia with Love (1963)"                                      => 729
  "House II: The Second Story (1987)"                                 => 1247
  "Held Up (2000)"                                                    => 3549
  "Missing in Action 2: The Beginning (1985)"                         => 2177
  "Murder, My Sweet (1944)"                                           => 996
  "Hidden, The (1987)"                                                => 981
  "Cable Guy, The (1996)"                                             => 669
  "Big Kahuna, The (2000)"                                            => 893
  "Addams Family Values (1993)"                                       => 1857
  "Farinelli: il castrato (1994)"                                     => 1945
  "Education of Little T

In [19]:
# We can extract the titles ...
keys(rating_set.item_to_index)

Base.KeySet for a Dict{AbstractString,Int32} with 3706 entries. Keys:
  "Fried Green Tomatoes (1991)"
  "Milk Money (1994)"
  "From Russia with Love (1963)"
  "House II: The Second Story (1987)"
  "Held Up (2000)"
  "Missing in Action 2: The Beginning (1985)"
  "Murder, My Sweet (1944)"
  "Hidden, The (1987)"
  "Cable Guy, The (1996)"
  "Big Kahuna, The (2000)"
  "Addams Family Values (1993)"
  "Farinelli: il castrato (1994)"
  "Education of Little Tree, The (1997)"
  "In God's Hands (1998)"
  "Last Man Standing (1996)"
  "Sixth Sense, The (1999)"
  "Star Maps (1997)"
  "Girl, Interrupted (1999)"
  "Stand by Me (1986)"
  "Rob Roy (1995)"
  "Caligula (1980)"
  "Flirting With Disaster (1996)"
  "Hook (1991)"
  "Institute Benjamenta, or This Dream People Call Human Life (1995)"
  "Way We Were, The (1973)"
  ⋮

In [20]:
# or codes
values(rating_set.item_to_index)

Base.ValueIterator for a Dict{AbstractString,Int32} with 3706 entries. Values:
  594
  1361
  729
  1247
  3549
  2177
  996
  981
  669
  893
  1857
  1945
  2759
  2814
  1840
  39
  2670
  32
  85
  449
  2532
  1290
  657
  3397
  762
  ⋮

In [23]:
# Which movies did the user "3000" grade?
user_ratings(rating_set, "3000")

91-element Array{Tuple{Pair{Int32,SubString{String}},Float32},1}:
 (3176 => "Goodbye, Lover (1999)", 5.0)
 (2177 => "Missing in Action 2: The Beginning (1985)", 5.0)
 (530 => "Sling Blade (1996)", 5.0)
 (3283 => "Jamaica Inn (1939)", 5.0)
 (594 => "Fried Green Tomatoes (1991)", 5.0)
 (2611 => "Dersu Uzala (1974)", 5.0)
 (772 => "Jerk, The (1979)", 5.0)
 (802 => "Holy Man (1998)", 5.0)
 (1071 => "Last Emperor, The (1987)", 5.0)
 (2051 => "Bread and Chocolate (Pane e cioccolata) (1973)", 5.0)
 (2861 => "Tales of Terror (1962)", 5.0)
 (2058 => "Eye of the Beholder (1999)", 5.0)
 (1194 => "High School High (1996)", 4.0)
 ⋮
 (2246 => "Tumbleweeds (1999)", 2.0)
 (550 => "Hype! (1996)", 2.0)
 (3080 => "Vibes (1988)", 2.0)
 (454 => "Postino, Il (The Postman) (1994)", 2.0)
 (3243 => "Turn It Up (2000)", 2.0)
 (2247 => "Dear Diary (Caro Diario) (1994)", 1.0)
 (126 => "Con Air (1997)", 1.0)
 (1449 => "It Happened One Night (1934)", 1.0)
 (519 => "Time to Kill, A (1996)", 1.0)
 (1314 => "Stripes (

In [24]:
# Let us find the exact title and code for "Blade Runner"
for k in keys(rating_set.item_to_index)
    if occursin("Blade",k)
        println(k)
    end
end

Sling Blade (1996)
Blade (1998)
Blade Runner (1982)
Some Folks Call It a Sling Blade (1993)


In [19]:
# Did the user "3000" grade "Blade Runner" ?
for k in user_ratings(rating_set,"3000")
    if occursin("Blade",k[1][2])
        println(k)
    end
end

In [25]:
# How did the user "3000" grade "Sling Blade" ?
for k in user_ratings(rating_set,"3000")
    if occursin("Blade",k[1][2])
        println(k)
    end
end

(530 => "Sling Blade (1996)", 5.0f0)


In [21]:
get(rating_set.item_to_index,"Blade Runner (1982)",0)

744

In [26]:
# This takes about half a minute
model = train(rating_set, 25);

Computing truncated rank 25 SVD 100%|███████████████████| Time: 0:00:17


In [27]:
propertynames(model)

(:user_to_index, :item_to_index, :U, :S, :V)

In [28]:
model.U

6040×25 Array{Float32,2}:
 0.011064    0.0121326    0.0165449    0.00843277  …   0.0179598   0.0133953
 0.0141649   0.0109572    0.0102553    0.0102382       0.00935903  0.0133421
 0.0101057   0.00901163   0.0166677    0.00849275      0.01271     0.0157814
 0.00715001  0.00672993   0.0156254    0.00426465      0.00966728  0.0137939
 0.013589    0.0155196    0.00188872   0.0147016       0.010734    0.00745202
 0.0108929   0.0169884    0.0159084    0.0156219   …   0.00926392  0.0148474
 0.00857805  0.00759389   0.0189076    0.00461746      0.00895364  0.0161191
 0.015564    0.0145136    0.00940196   0.0133166       0.00773994  0.0125608
 0.0133559   0.00869898   0.0112573    0.00714651      0.00736336  0.00906555
 0.0187443   0.0123034   -0.0105554    0.0242374       0.00484169  0.0150427
 0.0129478   0.0113956    0.00997051   0.0108549   …   0.00388393  0.00994879
 0.00578818  0.00825223   0.0134788    0.0067715       0.0105869   0.019237
 0.012383    0.00982242   0.0130446    0.0076709

In [29]:
model.S

25-element Array{Float32,1}:
 8190.9375
 2148.8425
  829.60046
 1148.2534
  840.6759
 1160.3884
  621.52594
  389.60126
  375.55692
  274.3554
  242.55199
  333.36444
  309.44574
  224.46367
  271.1998
  339.79025
  344.26297
  187.46455
  200.84787
  194.18845
  170.72044
  140.97693
  136.65704
  107.50458
   89.49921

In [30]:
model.V

3706×25 Array{Float32,2}:
 0.0357976   0.0078135   0.0673247   …  -0.00666704    0.00384758
 0.0250467   0.014368    0.0196545      -0.00641384    0.0049989
 0.0305158   0.0130995   0.0310579      -0.00418702    0.0189321
 0.0319803   0.011746    0.053455        0.0114667     0.0026196
 0.0316291   0.00758929  0.0557559       0.00920958    0.00201769
 0.0344305   0.00457543  0.0706544   …  -0.00972814   -0.00534585
 0.0303098   0.00825883  0.0287886       0.00584633    0.00444616
 0.0330673   0.00402261  0.0524465       0.0123761     0.00817346
 0.0289681   0.00720155  0.022626        0.00612513    0.020826
 0.0331869   0.00479188  0.0543633      -0.0162178     0.0351095
 0.0300653   0.00524444  0.037441    …  -0.000832079   0.00155316
 0.0174143   0.0268644   0.0093041       0.00805911    0.0133764
 0.0259354   0.018915    0.0138953       0.00935358    0.00497114
 ⋮                                   ⋱                
 0.00119165  0.00261155  0.00407048      0.0144404     0.0154552
 0.

In [38]:
similar_items(model, "Friday the 13th (1980)",max_results=20)

20-element Array{SubString{String},1}:
 "Friday the 13th (1980)"
 "Amityville Horror, The (1979)"
 "Cujo (1983)"
 "Pet Sematary (1989)"
 "Halloween II (1981)"
 "Jaws 2 (1978)"
 "Omen, The (1976)"
 "Nightmare on Elm Street Part 2: Freddy's Revenge, A (1985)"
 "Friday the 13th Part 3: 3D (1982)"
 "Friday the 13th: The Final Chapter (1984)"
 "Stepford Wives, The (1975)"
 "First Blood (1982)"
 "Friday the 13th Part 2 (1981)"
 "Nightmare on Elm Street 5: The Dream Child, A (1989)"
 "Howling, The (1980)"
 "Children of the Corn (1984)"
 "Creepshow (1982)"
 "Damien: Omen II (1978)"
 "Porky's (1981)"
 "Christine (1983)"

In [37]:
# Take a look at the function
@which similar_items(model, "Friday the 13th (1980)")

(::IncrementalSVD.var"#similar_items##kw")(::Any, ::typeof(similar_items), model::RatingsModel, item) in IncrementalSVD at C:\Users\Ivan_Slapnicar\.julia\packages\IncrementalSVD\8lAfd\src\util.jl:113

In [39]:
similar_items(model, "Citizen Kane (1941)")

10-element Array{SubString{String},1}:
 "Citizen Kane (1941)"
 "Chinatown (1974)"
 "M*A*S*H (1970)"
 "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)"
 "Boat, The (Das Boot) (1981)"
 "Rear Window (1954)"
 "2001: A Space Odyssey (1968)"
 "Casablanca (1942)"
 "Vertigo (1958)"
 "Lawrence of Arabia (1962)"

In [41]:
similar_users(model,"3000",max_results=20)

20-element Array{SubString{String},1}:
 "3000"
 "4501"
 "1360"
 "5613"
 "5715"
 "5619"
 "4323"
 "347"
 "514"
 "5572"
 "260"
 "120"
 "882"
 "3361"
 "3479"
 "2202"
 "1222"
 "3183"
 "1430"
 "1458"

In [42]:
# What is the opinion of user "3000" about "Blade Runner (1982)" 
# in the approximate model (no true mark) ?
get_predicted_rating(model, "3000", "Blade Runner (1982)")

4.197244f0

In [43]:
# What is the opinion of user "3000" about "Citizen Kane (1941)"
# (no true mark!) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Citizen Kane (1941)")

4.203371f0

In [44]:
# What is the opinion of user "3000" about "Sling Blade (1996)"
# in the approximate model (true mark 5.0) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Sling Blade (1996)")

4.061932f0

In [45]:
# What is the opinion of user "3000" about "Time to Kill, A (1996)")
# in the approximate model (true mark 1.0) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Time to Kill, A (1996)")

3.109532f0

## Thank you for you attention

### Questions?